In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import csv

In [2]:
def fileimport(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        raw_data = list(reader)
    features = raw_data.pop(0)
    features.remove("class")
    targets = [row.pop(0) for row in raw_data]

    test1 = len(raw_data[0]) == len(features)
    test2 = len(raw_data) == len(targets)
    if test1 and test2:
        return raw_data, targets, features
    else:
        print("ERROR")
        exit(-1)

In [3]:
def entropyH(prob):
    if prob != 0:
        entr = -prob * np.log2(prob)
        return entr
    else:
        return 0

In [4]:
def total_entropy(targets):
    
    diff = {}
    n = len(targets)
    for t in targets:
        if t not in diff:
            diff[t] = 1
        else:
            diff[t] += 1
    entropy = 0
    for t in diff:
        p = diff[t] / float(n)
        entropy += entropyH(p)

    return entropy

In [5]:
def infogain(data, classes, feature):
    gain = 0
    nData = len(data)

    # compute all different values in column feature
    fi_s = {}
    for row in data:
        if row[feature] not in fi_s.keys():
            fi_s[row[feature]] = 1
        else:
            fi_s[row[feature]] += 1

    for fi in fi_s.keys():
        fi_entropy = 0
        row_indx = 0
        newClasses = {}
        classCounts = 0
        for row in data:
            if row[feature] == fi:
                classCounts += 1
                if classes[row_indx] in newClasses.keys():
                    newClasses[classes[row_indx]] += 1
                else:
                    newClasses[classes[row_indx]] = 1
            row_indx += 1

        for aclass in newClasses.keys():
            fi_entropy += entropyH(float(newClasses[aclass]) / classCounts)

        gain += float(fi_s[fi]) / nData * fi_entropy
    return gain

In [6]:
def update(data, targets, feature, fi):
    
    new_data = []
    new_targets = []
    nFeatures = len(data[0])
    row_idx = 0
    
    for row in data:
        if row[feature] == fi:
            if feature == 0:
                new_row = row[1:]
            elif feature == nFeatures:
                new_row = row[:-1]
            else:
                new_row = row[:feature]
                new_row.extend(row[feature + 1:])

            new_data.append(new_row)
            new_targets.append(targets[row_idx])
        row_idx += 1

    return new_targets, new_data

In [7]:
def buildtree(data, classes, features):

    nData = len(data)
    nFeatures = len(features)
    # Have reached an empty branch
    uniqueT = {}
    for aclass in classes:
        if aclass in uniqueT.keys():
            uniqueT[aclass] += 1
        else:
            uniqueT[aclass] = 1

    default = max(uniqueT, key=uniqueT.get)
    if nData == 0 or nFeatures == 0:
        return default
    elif len(np.unique(classes)) == 1:
        # Only 1 class remains
        return classes[0]
    else:
        # Choose which feature is best
        totalEntropy = total_entropy(classes)
        gain = np.zeros(nFeatures)
        for feature in range(nFeatures):
            g = infogain(data, classes, feature)
            gain[feature] = totalEntropy - g
        best = np.argmax(gain)  # index of the best feature
        fi_s = np.unique(np.transpose(data)[best])
        feature = features.pop(best)    # Feature Name at that "best" position
        tree = {feature: {}}
        # Find the possible feature values
        for fi in fi_s:
            # Find the datapoints with each feature value
            t, d = update(data, classes, best, fi)
            # Now recurse to the next level
            subtree = buildtree(d, t, features)
            # And on returning, add the subtree on to the tree
            tree[feature][fi] = subtree
        return tree

In [8]:
def train(csv_file):
    data, targets, features = fileimport(filename=csv_file)
    tree = buildtree(data, targets, features)
    return tree

In [9]:
def tree_output(tree, data_row, features):
    if type(tree) is not dict:
        return str(tree)
    else:
        for key in tree.keys():
            f_idx = features.index(key)
            f_i = data_row[f_idx]
            return tree_output(tree[key][f_i], data_row, features)

In [12]:
def predictor(file, tree):

    with open(file, 'r') as f:
        reader = csv.reader(f)
        raw_data = list(reader)
        
    features = raw_data.pop(0)
    features.remove("class")
    targets = [row.pop(0) for row in raw_data]

    predval = 0
    n = len(raw_data)
    row_indx = 0
    for row in raw_data:
        output = tree_output(tree, row, features)
        if output == targets[row_indx]:
            
            predval += 1
        row_indx += 1
    
    return predval*100/float(n)

In [11]:
def main():
    train_file = "mushrooms_train_updated.csv"
    test_file = "mushrooms_test_updated.csv"
    
#    test_file = pd.read_csv('mushrooms_test_updated.csv')
#    train_file = pd.read_csv('mushrooms_train_updated.csv')

    tree = train(train_file)
    prediction_accuracy = predictor(test_file, tree)
    
#    with open("mushrooms_test_updated.csv", 'r') as f:
#        reader = csv.reader(f)
#        raw_data = list(reader)
#
#    predicted_values = predictval(test_file, tree)
    
    print("-------------------------------------------------------------------")
    pprint(tree)
    
    print("-------------------------------------------------------------------")
    print("Accuracy:", prediction_accuracy,"%")
    print("-------------------------------------------------------------------")
    
#    maxAccuracy = calculateAccuracy(test_file, tree)
#    bestTree = copy.deepcopy(tree)
#    countOfNodes = bestTree.countNodes(bestTree.root)
#    c = 0
#
#    while c < 10:    
#        c += 1
#        pruneNum = round(countOfNodes*pruneFactor)
#        pruneTree = Tree()
#        pruneTree = copy.deepcopy(bestTree)
#        postPruning(pruneNum,pruneTree.root)
#        temp = calculateAccuracy(dvalidation, pruneTree)
#        if temp > maxAccuracy:
#
#            maxAccuracy = temp
#            bestTree = copy.deepcopy(pruneTree)
#            countOfNodes = bestTree.countNodes(bestTree.root)
#    
#    print("Accuracy of the model on the training dataset = " , predictor(test_file, tree) , "%")

main()


-------------------------------------------------------------------
{'odor': {'a': 'e',
          'c': 'p',
          'f': 'p',
          'l': 'e',
          'm': 'p',
          'n': {'spore-print-color': {'b': 'e',
                                      'h': 'e',
                                      'k': 'e',
                                      'n': 'e',
                                      'o': 'e',
                                      'r': 'p',
                                      'w': {'habitat': {'d': {'gill-size': {'b': 'e',
                                                                            'n': 'p'}},
                                                        'g': 'e',
                                                        'l': {'cap-color': {'c': 'e',
                                                                            'n': 'e',
                                                                            'w': 'p',
                                              